Optical Flow nós dá a capacidade de detectar o movimento de objetos em uma sequência de frames, admitindo que:
 - a intensidade dos pixels se mantêm constante ao longo do tempo;
 - existe uma vizinhança de pixels que se movem juntos.
 
Essa técnica nos dá a capacidade de descobrir direção e amplitude do movimento. Isso é bastante interessante! Pense na gama de aplicações em que tais informações podem ser uteis. Você pode, por exemplo, prever quando um impacto irá ocorrer em um objeto alvo, ou economizar processamento do seu sistema analisando somente objetos que se movem na direção do seu interesse, ou estabilizar seu stream.

O OpenCV possui dois tipos de algoritmos de Optical Flow, e nós vamos focar aqui no Dense Optical Flow, ou seja, vamos obter a noção de movimento de toda a cena. Dê uma olhada depois no outro tipo de detecção, onde são escolhidas apenas algumas features da imagem, e essas features são então trackeadas. Sem mais delongas, vamos para o código! Segue abaixo.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import mode
from argparse import ArgumentParser

In [6]:

directions_map = np.zeros([10, 5])

cap = cv.VideoCapture(0)



plt.ion()
# Bom, como o Optical Flow calcula os vetores de movimento provenientes de dois frames consecutivos, 
# o primeiro passo é armazenar um frame inicial
frame_previous = cap.read()[1]
gray_previous = cv.cvtColor(frame_previous, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame_previous)
hsv[:, :, 1] = 255
param = {
    'pyr_scale': 0.5,
    'levels': 3,
    'winsize':15,
    'iterations': 3,
    'poly_n': 5,
    'poly_sigma': 1.1,
    'flags': cv.OPTFLOW_LK_GET_MIN_EIGENVALS
}

while True:
    grabbed, frame = cap.read()
    if not grabbed:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    #  O Dense Optical Flow é calculado através da função calcOpticalFlowFarnebac
    # Essa função retorna um ndarray com o mesmo número de linhas e colunas que as imagens utilizadas para o cálculo, 
    # porém com dois canais: o primeiro com as coordenadas em x, e o segundo, em y.
    flow = cv.calcOpticalFlowFarneback(gray_previous, gray, None, **param)
    # No OpenCV, nós podemos utilizar a função cartToPolar para então termos a magnitude e sentido (ângulo) do movimento através das coordenadas anteriores, linha 61 do código. Reparem no argumento angleInDegrees=True. Se False (default), ele retorna em radianos. Em graus, as direções são:
    # puramente para esquerda — 0 / 360°;
    # puramente para baixo — 90°;
    # puramente para a direita — 180º;
    # puramente para cima — 270°.
    mag, ang = cv.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees=True)
    ang_180 = ang/2
    
    # armazenamos o frame atual como o frame que será utilizado para o cálculo no frame seguinte
    gray_previous = gray
    
    # filtramos somente as direções onde o movimento apresentava uma magnitude maior que um limiar 
    # (no caso 10). Isso serve para retirar alguns ruídos de movimentos que, na verdade, não são movimentos,
    # e sim uma pequena mudança nas intensidades dos pixels.
    move_sense = ang[mag > 10]
    move_mode = mode(move_sense)[0]

    if 10 < move_mode <= 100:
        directions_map[-1, 0] = 1
        directions_map[-1, 1:] = 0
        directions_map = np.roll(directions_map, -1, axis=0)
    elif 100 < move_mode <= 190:
        directions_map[-1, 1] = 1
        directions_map[-1, :1] = 0
        directions_map[-1, 2:] = 0
        directions_map = np.roll(directions_map, -1, axis=0)
    elif 190 < move_mode <= 280:
        directions_map[-1, 2] = 1
        directions_map[-1, :2] = 0
        directions_map[-1, 3:] = 0
        directions_map = np.roll(directions_map, -1, axis=0)
    elif 280 < move_mode or move_mode < 10:
        directions_map[-1, 3] = 1
        directions_map[-1, :3] = 0
        directions_map[-1, 4:] = 0
        directions_map = np.roll(directions_map, -1, axis=0)
    else:
        directions_map[-1, -1] = 1
        directions_map[-1, :-1] = 0
        directions_map = np.roll(directions_map, 1, axis=0)



    loc = directions_map.mean(axis=0).argmax()
    if loc == 0:
        text = 'Movendo para baixo'
    elif loc == 1:
        text = 'Movendo para a direita'
    elif loc == 2:
        text = 'Movendo para cima'
    elif loc == 3:
        text = 'Movendo para a esquerda'
    else:
        text = 'SEM MOVIMENTO'

    hsv[:, :, 0] = ang_180
    hsv[:, :, 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    rgb = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    frame = cv.flip(frame, 1)
    cv.putText(frame, text, (30, 90), cv.FONT_HERSHEY_COMPLEX, frame.shape[1] / 500, (0, 0, 255), 2)

    k = cv.waitKey(1) & 0xff
    if k == ord('q'):
        break

    cv.imshow('Mask', rgb)
    cv.imshow('Frame', frame)
    k = cv.waitKey(1) & 0xff
    if k == ord('q'):
        break

cap.release()


plt.ioff()
cv.destroyAllWindows()

error: OpenCV(4.2.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
